## Lab Assignment One: Exploring Table Data

#### Luis Garduno

### 1. Business Understanding

##### About League of Legends :
Developed by Riot Studios, League of Legends, or "LoL", is an online multiplayer
video-game that is available to Windows/MacOS users. LoL consists 2 teams ('Blue
& 'Red') facing each other, where the main objective is to destroy the opposing
teams home base while facing obstacles throughout the way. Perks & gold are able
to be obtained by players/teams through completing tasks such as eliminating
players, enemy creeps, or dragons. Players then spend the gold to purchase items
that help deal more damage towards enemies.

League of Legends offers different game modes, such as ranked. A ranked game on
average lasts 30-45 minutes. In this game mode, players are given a rank based
off of the number of wins + the number of games played. "Diamond" is one of the
highest ranks a player may obtain and is known to be extremely competitive. The
dataset contains the first 10 minute analytics of each team for different
diamond ranked matches.

The dataset contains the first 10 minute analytics of each team for different
diamond ranked matches. Each team has 19 features (38 total) that include
objectives such as creep score (CS), obtained dragons, total number of kills,
etc. As for the instances, there is a total of 9,879 different diamond ranked matches.

Once the data is analyzed, teams/players would be able to conceptualize the
level of priority different features/objectives have during early stages of
diamond ranked matches. They could then use this information to adjust their
strategy to one proven to win matches.

-------------------------------------

Dataset : https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min

Question Of Interest : As of the first 10 minutes, which team will win?

### 3. Data Visualization
// [2 points] Visualize basic feature distributions.
// 1. That is, plot the dynamic range and exploratory distribution plots (like boxplots, histograms, kernel density estimation) to better understand the data.
// 2. Describe anything meaningful or potentially useful you discover from these visualizations.
// 3. These may also help to understand what data is missing or needs imputation.
// 4. Note: You can also use data from other sources to bolster visualizations.
// 5. Visualize at least five plots, at least one categorical and at least one numeric.

// [2.5 points]
// 1. Ask 3 interesting questions that are relevant to your dataset & explore visuals that help answer these questions.
// 2. Use whichever visualization method is appropriate for your data.
// 3. Important: Interpret the implications for each visualization.
#### 3.1 Data Exploration
#### 3.2 Data Relationship Exploration

### 4. Dimensionality Reduction
#### Reference

### 2. Data Understanding
// [1.5 points]
// 1. Appropriately define data types.
// 2. What data type should be used to represent each data attribute?
// 3. For datasets w/ a large number of attributes, only discuss a subset of relevant attributes.

// [1.5 points] Verify data quality:
// 1. Explain any missing values or duplicate data.
// 2. Visualize entries that are missing/complete for different attributes.
// 3. Are those mistakes?
// 4. Why do these quality issues exist in the data?
// 5. How do you deal with these problems?
// 6. Give justifications for your methods (elimination or imputation).

In [30]:
import numpy as np
import pandas as pd

# Load in the dataset into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/luisegarduno/MachineLearning_Projects/master/Datasets/high_diamond_ranked_10min.csv')

# delete gameID column
del df['gameId']

# Dimensions for dataframe
print(df.shape)

df.head()

(9879, 39)


,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,0,12,1,0,5,5,5,0,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,0,15,0,0,7,11,4,1,1,0,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,0,43,1,0,4,5,5,1,0,1,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,0,75,4,0,6,6,6,0,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


By printing out dimensions of the dataframe we able to see that there are a
total of 9,879 instances, and 39 instances. The "gameId" column is then
removed since it will have no impact on the outcome.

When outputting the dataframe head, we are able to identify that there are 19
attributes for each the blue & red team. Features for each team includes :
- Wards placed & destroyed
- Total number of kills, deaths, & assists
- First Bloods (1st elimination of the game)
- Total : towers destroyed, gold, experience
- Average : level, CS per minute, & gold per minute
- Difference in gold & experience between the teams
- Objective kills : elite monsters, dragons, heralds, minions, & jungle minions

The winner of the match is also included as one of the features, listed as
"blueWins" where '1' means blue team won, & '0' meaning the red team won.